In [9]:
# !pip install tensorboard

In [7]:
import keras_tuner as kt
import tensorflow as tf

In [10]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [11]:
#
# Import Keras modules
#
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.utils import to_categorical

In [12]:
from datetime import datetime
now = datetime.now()
logdir_tuner = "test_logs_/" + now.strftime("%Y%m%d-%H%M%S") + "/" + "Tuner"
logdir_tensorboard = "test_logs_/" + now.strftime("%Y%m%d-%H%M%S") + "/" + "TensorBoard"

In [13]:
#
# Load the iris dataset
#
iris = datasets.load_iris()
X = iris.data
y = iris.target
#

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [6]:
# def _model(hp):
#     model = tf.keras.Sequential([
#         layers.Dense(
#             hp.Int("dense_1_units", min_value=4, max_value=12, step=4, default=8),
#             input_shape=(X_train.shape[1],)
            
#         ),
#         layers.Dense(
#             hp.Int("dense_2_units", min_value=4, max_value=12, step=4, default=8), 
#             activation="relu"
#         ),
#         layers.Dense(3, activation="softmax"),
#     ])

#     model.compile(
#         optimizer=tf.keras.optimizers.Adam(
#             hp.Choice("learning_rate", values=[1e-2, 1e-3])
#         ),
#         loss="categorical_crossentropy",
#         metrics=["accuracy"]
#     )
#     return model

In [14]:
import numpy as np
import pandas as pd
pd.Series(y_train).unique().size

3

In [15]:
def classification_model(hp):
    inputs = Input(shape = X_train.shape[1])
    hidden = inputs
    
    hp_layers = hp.Int("Dense_layers", min_value=1, max_value=3, step=1, default=3)
    hp_dropout = hp.Choice("Dropout_layer", [0.1, 0.2, 0.3], default=0.2)
        
    for i in range(hp_layers):
        hidden = layers.Dense(hp.Choice("dense_units_" + str(i), [16, 32, 64], default=16),
                              activation=hp.Choice("activation_" + str(i), ['relu', 'tanh']))(hidden)
        hidden = layers.Dropout(hp_dropout)(hidden)
    
    outputs = layers.Dense(pd.Series(y_train).unique().size, 
                           activation="softmax")(hidden)
    
    model = tf.keras.Model(inputs, outputs)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            hp.Choice("learning_rate", values=[1e-2, 1e-3])
        ),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

In [16]:
tuner = kt.RandomSearch(
    classification_model,
    objective="val_accuracy",
    max_trials=10,
    overwrite=True,
    directory=logdir_tuner,
    project_name="keras-tuner-test",
)

In [17]:
tuner.search_space_summary()

Search space summary
Default search space size: 9
Dense_layers (Int)
{'default': 3, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': None}
Dropout_layer (Choice)
{'default': 0.2, 'conditions': [], 'values': [0.1, 0.2, 0.3], 'ordered': True}
dense_units_0 (Choice)
{'default': 16, 'conditions': [], 'values': [16, 32, 64], 'ordered': True}
activation_0 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
dense_units_1 (Choice)
{'default': 16, 'conditions': [], 'values': [16, 32, 64], 'ordered': True}
activation_1 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
dense_units_2 (Choice)
{'default': 16, 'conditions': [], 'values': [16, 32, 64], 'ordered': True}
activation_2 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001], 'ordered': True}


In [18]:
tuner.search(
    X_train[:,:], 
    to_categorical(y_train), 
    epochs=5, 
    validation_data=(X_test[:,:], to_categorical(y_test)),
    callbacks=[tf.keras.callbacks.TensorBoard(logdir_tensorboard)]
)

Trial 10 Complete [00h 00m 00s]
val_accuracy: 0.8666666746139526

Best val_accuracy So Far: 0.9777777791023254
Total elapsed time: 00h 00m 09s
INFO:tensorflow:Oracle triggered exit


In [19]:
tuner.results_summary()

Results summary
Results in test_logs_/20220424-013354/Tuner\keras-tuner-test
Showing 10 best trials
Trial summary
Hyperparameters:
Dense_layers: 1
Dropout_layer: 0.2
dense_units_0: 32
activation_0: tanh
dense_units_1: 16
activation_1: tanh
dense_units_2: 16
activation_2: relu
learning_rate: 0.01
Score: 0.9777777791023254
Trial summary
Hyperparameters:
Dense_layers: 3
Dropout_layer: 0.2
dense_units_0: 32
activation_0: tanh
dense_units_1: 32
activation_1: tanh
dense_units_2: 16
activation_2: tanh
learning_rate: 0.001
Score: 0.9777777791023254
Trial summary
Hyperparameters:
Dense_layers: 2
Dropout_layer: 0.1
dense_units_0: 32
activation_0: tanh
dense_units_1: 32
activation_1: relu
dense_units_2: 32
activation_2: relu
learning_rate: 0.01
Score: 0.9555555582046509
Trial summary
Hyperparameters:
Dense_layers: 3
Dropout_layer: 0.2
dense_units_0: 64
activation_0: relu
dense_units_1: 64
activation_1: tanh
dense_units_2: 64
activation_2: tanh
learning_rate: 0.01
Score: 0.9333333373069763
Trial s

In [20]:
model = tuner.get_best_models(num_models=1)[0]

In [21]:
y_pred = np.argmax(model.predict(X_test), axis=-1)
accuracy = np.sum(y_pred == y_test) / len(y_test) * 100
accuracy

97.77777777777777

In [22]:
import tensorboard

In [23]:
%load_ext tensorboard

In [24]:
logdir_tensorboard

'test_logs_/20220424-013354/TensorBoard'

In [18]:
tuner.get_best_hyperparameters()[0]

In [25]:
# tensorboard --inspect --logdir 'logdir_tensorboard'


In [26]:
%tensorboard --logdir "$logdir_tensorboard"